<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# PySpark Huggingface Inferencing
### Sentence Transformers with PyTorch

In this notebook, we demonstrate distributed inference with the Huggingface SentenceTransformer library for sentence embedding.  
From: https://huggingface.co/sentence-transformers

In [2]:
import torch
from sentence_transformers import SentenceTransformer

# Manually enable Huggingface tokenizer parallelism to avoid disabling with PySpark parallelism.
# See (https://github.com/huggingface/transformers/issues/5486) for more info. 
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer("paraphrase-MiniLM-L6-v2", device=device)

sentence = ['This framework generates embeddings for each input sentence']
embedding = model.encode(sentence)

In [3]:
print(embedding[0][:20])

[-0.17621444  0.1206013  -0.29362372 -0.22985819 -0.08229247  0.2377093
  0.33998525 -0.7809643   0.11812777  0.16337365 -0.13771524  0.24028276
  0.4251256   0.17241786  0.10527937  0.5181643   0.062222    0.39928585
 -0.18165241 -0.58557856]


## PySpark

In [4]:
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf, col, struct
from pyspark.ml.functions import predict_batch_udf

In [5]:
import json
import pandas as pd
import datasets
from datasets import load_dataset
datasets.disable_progress_bars()

Check the cluster environment to handle any platform-specific Spark configurations.

In [6]:
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_IMAGE_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [7]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/02/04 13:40:01 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/02/04 13:40:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 13:40:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Load the IMBD Movie Reviews dataset from Huggingface.

In [8]:
dataset = load_dataset("imdb", split="test")
dataset = dataset.to_pandas().drop(columns="label")

#### Create PySpark DataFrame

In [9]:
df = spark.createDataFrame(dataset).repartition(8)
df.schema

StructType([StructField('text', StringType(), True)])

In [10]:
df.count()

25000

In [11]:
df.take(1)

25/02/04 13:40:08 WARN TaskSetManager: Stage 6 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


[Row(text="Anyone remember the first CKY, CKY2K etc..? Back when it was about making crazy cool stuff, rather than watching Bam Margera act like a douchebag, spoiled 5 year old, super/rock-star wannabe.<br /><br />The show used to be awesome, however, Bam's fame and wealth has led him to believe, that we now enjoy him acting childish and idiotic, more than actual cool stuff, that used to be in ex. CKY2K.<br /><br />The acts are so repetitive, there's like nothing new, except annoying stupidity and rehearsed comments... The only things we see is Bam Margera, so busy showing us how much he doesn't care, how much money he got or whatsoever.<br /><br />I really got nothing much left to say except, give us back CKY2K, cause Bam suck..<br /><br />I enjoy watching Steve-o, Knoxville etc. a thousand times more.")]

In [12]:
data_path = "spark-dl-datasets/imdb_test"
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/spark-dl-datasets")
    data_path = "dbfs:/FileStore/" + data_path

df.write.mode("overwrite").parquet(data_path)

25/02/04 13:40:08 WARN TaskSetManager: Stage 9 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


#### Load and preprocess DataFrame

Define our preprocess function. We'll take the first sentence from each sample as our input for translation.

In [13]:
@pandas_udf("string")
def preprocess(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

In [14]:
# Limit to N rows, since this can be slow
df = spark.read.parquet(data_path).limit(256).repartition(8)
df = df.select(preprocess(col("text")).alias("input")).cache()
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               input|
+----------------------------------------------------------------------------------------------------+
|Doesn't anyone bother to check where this kind of sludge comes from before blathering on about it...|
|                          There were two things I hated about WASTED : The directing and the script |
|                                I'm rather surprised that anybody found this film touching or moving|
|Cultural Vandalism Is the new Hallmark production of Gulliver's Travels an act of cultural vandal...|
|I was at Wrestlemania VI in Toronto as a 10 year old, and the event I saw then was pretty differe...|
|                                                                     This movie has been done before|
|[ as a new resolution for this year 2005, i decide to write a comment fo

## Inference using Spark DL API

Distributed inference using the PySpark [predict_batch_udf](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.functions.predict_batch_udf.html#pyspark.ml.functions.predict_batch_udf):

- predict_batch_fn uses PyTorch APIs to load the model and return a predict function which operates on numpy arrays 
- predict_batch_udf will convert the Spark DataFrame columns into numpy input batches for the predict function

In [15]:
def predict_batch_fn():
    import torch
    from sentence_transformers import SentenceTransformer

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2", device=device)
    def predict(inputs):
        return model.encode(inputs.tolist())
    return predict

In [16]:
encode = predict_batch_udf(predict_batch_fn,
                           return_type=ArrayType(FloatType()),
                           batch_size=32)

In [17]:
%%time
# first pass caches model/fn
embeddings = df.withColumn("embedding", encode(struct("input")))
results = embeddings.collect()

CPU times: user 10.6 ms, sys: 4.83 ms, total: 15.4 ms
Wall time: 4.23 s


In [18]:
%%time
embeddings = df.withColumn("embedding", encode("input"))
results = embeddings.collect()

CPU times: user 6.7 ms, sys: 2.44 ms, total: 9.15 ms
Wall time: 163 ms


In [19]:
%%time
embeddings = df.withColumn("embedding", encode(col("input")))
results = embeddings.collect()

CPU times: user 5.37 ms, sys: 2.73 ms, total: 8.1 ms
Wall time: 232 ms


In [20]:
embeddings.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             input|                                         embedding|
+--------------------------------------------------+--------------------------------------------------+
|Doesn't anyone bother to check where this kind ...|[0.118947476, -0.053823642, -0.29726124, 0.0720...|
|There were two things I hated about WASTED : Th...|[0.18953452, 0.11079162, 0.07503566, 0.01050696...|
|I'm rather surprised that anybody found this fi...|[-0.0010759671, -0.14203517, -0.06649738, 0.129...|
|Cultural Vandalism Is the new Hallmark producti...|[0.34815887, -0.2966917, -0.10905265, 0.1051652...|
|I was at Wrestlemania VI in Toronto as a 10 yea...|[0.45902696, 0.019472413, 0.28720972, -0.070724...|
|                   This movie has been done before|[-0.062292397, -0.025909504, -0.031942524, 0.01...|
|[ as a new resolution for this year 2005, i dec...|[0.3469342, 

## Using Triton Inference Server
In this section, we demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-server.png" alt="drawing" width="700"/>

In [21]:
from functools import partial

Import the helper class from server_utils.py:

In [22]:
sc.addPyFile("server_utils.py")

from server_utils import TritonServerManager

Define the Triton Server function:

In [23]:
def triton_server(ports):
    import time
    import signal
    import numpy as np
    import torch
    from sentence_transformers import SentenceTransformer
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing sentence transformer on worker {TaskContext.get().partitionId()}.")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SentenceTransformer("paraphrase-MiniLM-L6-v2", device=device)
    print(f"SERVER: Using {device} device.")

    @batch
    def _infer_fn(**inputs):
        sentences = np.squeeze(inputs["text"])
        print(f"SERVER: Received batch of size {len(sentences)}")
        decoded_sentences = [s.decode("utf-8") for s in sentences]
        embeddings = model.encode(decoded_sentences)
        return {
            "embeddings": embeddings,
        }

    workspace_path = f"/tmp/triton_{time.strftime('%m_%d_%M_%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="SentenceTransformer",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="text", dtype=object, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="embeddings", dtype=np.float32, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=64,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def _stop_triton(signum, frame):
            # The server manager sends SIGTERM to stop the server; this function ensures graceful cleanup.
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, _stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

#### Start Triton servers

The `TritonServerManager` will handle the lifecycle of Triton server instances across the Spark cluster:
- Find available ports for HTTP/gRPC/metrics
- Deploy a server on each node via stage-level scheduling
- Gracefully shutdown servers across nodes

In [25]:
model_name = "SentenceTransformer"
server_manager = TritonServerManager(model_name=model_name)

In [ ]:
# Returns {'hostname', (server_pid, [http_port, grpc_port, metrics_port])}
server_manager.start_servers(triton_server)

2025-02-07 11:03:44,809 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-07 11:03:44,810 - INFO - Starting 1 servers.


{'cb4ae00-lcedt': (2020631, [7000, 7001, 7002])}

#### Define client function

Get the hostname -> url mapping from the server manager:

In [ ]:
host_to_http_url = server_manager.host_to_http_url  # or server_manager.host_to_grpc_url

In [28]:
def triton_fn(model_name, host_to_url):
    import socket
    import numpy as np
    from pytriton.client import ModelClient

    url = host_to_url[socket.gethostname()]
    print(f"Connecting to Triton model {model_name} at {url}.")

    def infer_batch(inputs):
        with ModelClient(url, model_name, inference_timeout_s=240) as client:
            flattened = np.squeeze(inputs).tolist()
            # Encode batch
            encoded_batch = [[text.encode("utf-8")] for text in flattened]
            encoded_batch_np = np.array(encoded_batch, dtype=np.bytes_)
            # Run inference
            result_data = client.infer_batch(encoded_batch_np)
            return result_data["embeddings"]
        
    return infer_batch

In [31]:
encode = predict_batch_udf(partial(triton_fn, model_name=model_name, host_to_url=host_to_http_url),
                           return_type=ArrayType(FloatType()),
                           input_tensor_shapes=[[1]],
                           batch_size=32)

#### Load and preprocess DataFrame

In [29]:
@pandas_udf("string")
def preprocess(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

In [30]:
df = spark.read.parquet(data_path).limit(256).repartition(8)
df = df.select(preprocess(col("text")).alias("input")).cache()

25/02/04 13:40:22 WARN CacheManager: Asked to cache already cached data.


#### Run Inference

In [32]:
%%time
# first pass caches model/fn
embeddings = df.withColumn("embedding", encode(struct("input")))
results = embeddings.collect()

CPU times: user 5.59 ms, sys: 5.1 ms, total: 10.7 ms
Wall time: 605 ms


In [33]:
%%time
embeddings = df.withColumn("embedding", encode("input"))
results = embeddings.collect()

CPU times: user 2.57 ms, sys: 4.36 ms, total: 6.93 ms
Wall time: 161 ms


In [34]:
%%time
embeddings = df.withColumn("embedding", encode(col("input")))
results = embeddings.collect()

CPU times: user 7.06 ms, sys: 605 μs, total: 7.67 ms
Wall time: 191 ms


In [35]:
embeddings.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                             input|                                         embedding|
+--------------------------------------------------+--------------------------------------------------+
|Doesn't anyone bother to check where this kind ...|[0.118947476, -0.053823642, -0.29726124, 0.0720...|
|There were two things I hated about WASTED : Th...|[0.18953452, 0.11079162, 0.07503566, 0.01050696...|
|I'm rather surprised that anybody found this fi...|[-0.0010759671, -0.14203517, -0.06649738, 0.129...|
|Cultural Vandalism Is the new Hallmark producti...|[0.34815887, -0.2966917, -0.10905265, 0.1051652...|
|I was at Wrestlemania VI in Toronto as a 10 yea...|[0.45902696, 0.019472413, 0.28720972, -0.070724...|
|                   This movie has been done before|[-0.062292397, -0.025909504, -0.031942524, 0.01...|
|[ as a new resolution for this year 2005, i dec...|[0.3469342, 

#### Stop Triton Server on each executor

In [36]:
server_manager.stop_servers()

2025-02-04 13:40:23,196 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-04 13:40:28,390 - INFO - Sucessfully stopped 1 servers.                 


[True]

In [37]:
if not on_databricks: # on databricks, spark.stop() puts the cluster in a bad state
    spark.stop()